<a href="https://colab.research.google.com/github/23subbhashit/IIITB-ML/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/ML IIITB/train.csv')
df = df.dropna(subset=['rating', 'review_text'])

In [20]:
from bs4 import BeautifulSoup
import unicodedata
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
from imblearn.over_sampling import RandomOverSampler
import pandas as pd


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet



def preprocess_text(text):

    text = text.lower() # Convert to lowercase


    text = BeautifulSoup(text, "html.parser").get_text() # Remove HTML tags

    # Remove numbers, punctuation, and special characters
    text = re.sub(r'\d+', '', text)
    text = text.translate(str.maketrans('', '', string.punctuation))


    words = word_tokenize(text) # Tokenization (split the text into words)


    stop_words = set(stopwords.words('english')) # Remove stop words
    words = [word for word in words if word not in stop_words]


    lemmatizer = WordNetLemmatizer() # Lemmatization (using WordNet Lemmatizer)
    words = [lemmatizer.lemmatize(word) for word in words]


    processed_text = ' '.join(words) # Join the words back into a single string

    return processed_text


tqdm.pandas()


df['review_text'] = df['review_text'].progress_apply(preprocess_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
  0%|          | 62/630000 [00:00<34:39, 302.93it/s]<ipython-input-20-4e973d915820>:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text() # Remove HTML tags
100%|██████████| 630000/630000 [19:50<00:00, 529.26it/s]


In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout



train_data, test_data = train_test_split(df[['review_text', 'rating']], test_size=0.2, random_state=42)


max_words = 5000  # Maximum number of words to keep
max_len = 100  # Maximum sequence length
embedding_dim = 100  # Embedding dimension

tokenizer = Tokenizer(num_words=max_words, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data['review_text'])
word_index = tokenizer.word_index
print("TOKENIZER START")
train_sequences = tokenizer.texts_to_sequences(train_data['review_text'])
test_sequences = tokenizer.texts_to_sequences(test_data['review_text'])
print("TOKENIZER END")
print("PADDING START")
X_train = pad_sequences(train_sequences, maxlen=max_len)
X_test = pad_sequences(test_sequences, maxlen=max_len)
print("PADDING END")

TOKENIZER START
TOKENIZER END
PADDING START
PADDING END


In [22]:
print("MODEL START")

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=max_len))
model.add(LSTM(128))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

MODEL START


In [23]:

y_train = train_data['rating'].values
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)


y_test = test_data['rating'].values
y_pred = model.predict(X_test)
predictions = [int(round(pred.argmax())) for pred in y_pred]


accuracy = accuracy_score(y_test, predictions)
print(f'Accuracy: {accuracy * 100:.2f}%')

Epoch 1/5
6300/6300 [==============================] - 1589s 252ms/step - loss: 1.1448 - accuracy: 0.5076 - val_loss: 1.0868 - val_accuracy: 0.5340
Epoch 2/5
6300/6300 [==============================] - 1657s 263ms/step - loss: 1.0618 - accuracy: 0.5447 - val_loss: 1.0709 - val_accuracy: 0.5400
Epoch 3/5
6300/6300 [==============================] - 1617s 257ms/step - loss: 1.0273 - accuracy: 0.5605 - val_loss: 1.0637 - val_accuracy: 0.5444
Epoch 4/5
6300/6300 [==============================] - 1618s 257ms/step - loss: 0.9972 - accuracy: 0.5747 - val_loss: 1.0702 - val_accuracy: 0.5424
Epoch 5/5
3938/3938 [==============================] - 232s 59ms/step
Accuracy: 54.62%


In [24]:
model.save('/content/drive/MyDrive/ML IIITB/MODEL1.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# TEST

In [25]:
test_df = pd.read_csv('/content/drive/MyDrive/ML IIITB/test.csv')
test_df = test_df.dropna(subset=['review_text'])
test_df.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,de3a6a28c83cda006b415d45d64674c9,1618,d76ce5becf493e5c653610edb806ffc4,"I'm going to keep this review short, because I...",Tue Jun 09 10:37:48 -0700 2015,Wed Jun 10 13:43:38 -0700 2015,Wed Jun 10 14:08:39 -0700 2015,Tue Jun 09 00:00:00 -0700 2015,8,0
1,d9cfab35a87e32084b1817dfb0e98748,15776309,9c5c9aed79255a1a610dfc153ee90ad6,"You know, I was really stoked to see this come...",Mon Apr 22 09:19:40 -0700 2013,Wed Jul 10 14:34:27 -0700 2013,Wed Jul 10 14:34:27 -0700 2013,Tue Jul 09 00:00:00 -0700 2013,0,0
2,24935a870a46525a37be92775ab18f76,478927,ad26332459cda8f40581fef7a29b800c,This is one of those books where you know you ...,Sat Mar 14 12:16:55 -0700 2015,Sun Apr 05 11:25:00 -0700 2015,Wed Mar 18 00:00:00 -0700 2015,Sat Mar 14 00:00:00 -0700 2015,14,0
3,6ccb40eabdd0db7895aac00963256469,22628,5bb75768be7f2ddcd632336778b67f5a,The perks of being a wallflower \n What does a...,Sat Sep 12 08:43:30 -0700 2015,Sat Sep 12 09:19:29 -0700 2015,NaN,NaN,0,0
4,9f9d0f6e9a6a5797a252ef81abc9421c,13596809,3d7f76ea566b9ce0700772236094d936,"So, I wrote a review for this when I read it b...",Sun Jul 14 19:36:13 -0700 2013,Thu Jun 02 16:45:45 -0700 2016,Sat Mar 23 00:00:00 -0700 2013,NaN,0,0


In [26]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270000 entries, 0 to 269999
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       270000 non-null  object
 1   book_id       270000 non-null  int64 
 2   review_id     270000 non-null  object
 3   review_text   270000 non-null  object
 4   date_added    270000 non-null  object
 5   date_updated  270000 non-null  object
 6   read_at       242459 non-null  object
 7   started_at    187740 non-null  object
 8   n_votes       270000 non-null  int64 
 9   n_comments    270000 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 20.6+ MB


In [27]:
tqdm.pandas()
test_df['review_text'] = test_df['review_text'].progress_apply(preprocess_text)

  0%|          | 35/270000 [00:00<12:58, 346.87it/s]<ipython-input-20-4e973d915820>:27: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text() # Remove HTML tags
100%|██████████| 270000/270000 [08:41<00:00, 517.31it/s]


In [28]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model


loaded_model = load_model('/content/drive/MyDrive/ML IIITB/MODEL1.h5')

max_words = 5000
max_len = 100


test_sequences = tokenizer.texts_to_sequences(test_df['review_text'])
X_test = pad_sequences(test_sequences, maxlen=max_len)

y_pred_prob = loaded_model.predict(X_test)
predictions = [int(round(pred.argmax())) for pred in y_pred_prob]

8438/8438 [==============================] - 510s 60ms/step


In [29]:
len(predictions)

270000

In [30]:
result_df = pd.DataFrame({
    'review_id': test_df['review_id'],
    'rating': predictions
})


result_df.to_csv('/content/drive/MyDrive/ML IIITB/predic22.csv', index=False, columns=["review_id", "rating"])